In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import nltk
import gensim
from gensim import corpora, models, similarities
nltk.download('punkt')
## NEED TO DOWNLOAD NTLK

[nltk_data] Error loading punkt: [Errno 60] Operation timed out


False

In [2]:
### load the meta data - USING DOWNLOADED DATASET

data = []
with gzip.open('reviews_Video_Games_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

231780
{'reviewerID': 'A2HD75EMZR8QLN', 'asin': '0700099867', 'reviewerName': '123', 'helpful': [8, 12], 'reviewText': 'Installing the game was a struggle (because of games for windows live bugs).Some championship races and cars can only be "unlocked" by buying them as an addon to the game. I paid nearly 30 dollars when the game was new. I don\'t like the idea that I have to keep paying to keep playing.I noticed no improvement in the physics or graphics compared to Dirt 2.I tossed it in the garbage and vowed never to buy another codemasters game. I\'m really tired of arcade style rally/racing games anyway.I\'ll continue to get my fix from Richard Burns Rally, and you should to. :)http://www.amazon.com/Richard-Burns-Rally-PC/dp/B000C97156/ref=sr_1_1?ie=UTF8&qid;=1341886844&sr;=8-1&keywords;=richard+burns+rallyThank you for reading my review! If you enjoyed it, be sure to rate it as helpful.', 'overall': 1.0, 'summary': "Pay to unlock content? I don't think so.", 'unixReviewTime': 134179

In [3]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))
df.columns

231780


Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [4]:
df.isna().sum()

reviewerID           0
asin                 0
reviewerName      2813
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [5]:
x = df.loc[:,'reviewText']
y = df.loc[:,'overall']
y_new = []
for i in range(y.size):
    if y[i] < 4:
        y_new.append('BAD')
    else:
        y_new.append('GOOD')

In [6]:
print(type(x[100]))
print(type(y_new[100]))

<class 'str'>
<class 'str'>


In [7]:
corpus = []
for i in range(x.size):
    corpus.append(" ".join([x[i], y_new[i]]))
df_new = pd.DataFrame(corpus)
df_new.head(5)

,0
0,Installing the game was a struggle (because of...
1,If you like rally cars get this game you will ...
2,1st shipment received a book instead of the ga...
3,"I got this version instead of the PS3 version,..."
4,I had Dirt 2 on Xbox 360 and it was an okay ga...


In [8]:
# https://stackabuse.com/implementing-word2vec-with-gensim-library-in-python/
# corpus2 = nltk.sent_tokenize(x)
all_words = [nltk.word_tokenize(sent) for sent in corpus]

In [9]:
word2vec = models.Word2Vec(all_words, min_count=2)

In [10]:
vocab = word2vec.wv.vocab
print(vocab)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
from gensim.models import KeyedVectors
# Store just the words + their trained embeddings.
word_vectors = word2vec.wv
word_vectors.save("word2vec.wordvectors")